# 1910-Powershell-as-a-Shell

## overview

- PowerShellと、今までのcmd.exeとの違いについて書く。

## jupyter notebook

https://nbviewer.jupyter.org/github/sakai-memoru/pwshnote/blob/master/1910-Powershell-as-a-Shell.ipynb

## オブジェクト指向型のシェル言語：Powershell

### オブジェクト指向によるソフトウェア
- 現代のソフトウェアでは、オブジェクト指向を基本的な考え方として取り入れて、ソフトウェアの仕組みが構成されている。

- システム開発で使われるオブジェクト指向のプログラミング言語として、Java、C#などが上げられる。最近のトレンドとしては、オブジェクト指向のスクリプト言語が、様々なシステム開発プロジェクトで使われている。オブジェクト指向のスクリプト言語として、RubyやPythonが上げられる。ちなみに、Javascriptも、プロトタイプをもとにオブジェクトを扱う、オブジェクト指向的なスクリプト言語である。

- PowerShellは、Pythonなどと同様にオブジェクト指向のスクリプト言語であり、かつ、Windows OSに対するCUIシェルで動作するシェル言語であると言える。


### オブジェクト指向OSとしてのWindows OS 

- オブジェクト指向の考え方は、OSにも取り入れられている。
- Windows OSは、Windows NTから、OS内部の構造は、オブジェクト指向で作られている。つまり、OS内部で管理している情報は、オブジェクト形式なのである。

- OSでは、コンピュータ資源（リソース）の管理を行い、コンピュータ上で実行されているプロセス、サービス等を制御し、実行の履歴（ログ）を記録する役割を持つ。このコンピュータ資源、プロセス(**process**)、サービス(**service**)、実行ログ(**EventLog**)、等を、Windows OS内部ではオブジェクトとして管理しているのである。

- PowerShellは、シェルスクリプトで、一連の操作を記述して、CUIシェル上で実行できる。つまりは、Windows OSが管理しているオブジェクトを取得して、操作することが可能となる。

### Windows OSが提供するShell

#### CUI Shell : cmd.exe

- 2006年より以前、Windows OSが提供していた標準CUI Shellは、cmd.exeである。コンソールより入力したコマンドを、OSが受付け、その操作結果を、テキストベースの情報として、返却する。

![Image from Gyazo](https://i.gyazo.com/860e8ca7fde0d47d7e9dfa917806f4ad.png)

- OS内部では、オブジェクトとして管理している情報を、cmd.exeでは、わざわざテキストベースの情報として表示している。
- OSが提供するコマンドを利用して、一連の動作を記述できるが、構造化プログラミング、オブジェクト指向言語に慣れてしまっているプログラマには、扱いにくい。

#### GUI Shell

- Windows OSは、GUIによるコンピュータ資源の操作の仕組みを提供している。Controll Panel, Explorer、Task Manager等の機能である。以下、Computer Setting画面より、Network情報を表示した例。

![Image from Gyazo](https://i.gyazo.com/f6741f5cecfbc3830c572e30b39ff83e.png)


- GUIは、直感的に操作しやすく、情報も見やすい。が、バッチ的に、大量に処理したい場合は、かえって手間がかかるという問題がある。管理者がPCの設定などを複数台、行いたい場合など、手作業による誤りなどの課題が発生することも多い。

#### オブジェクト指向型のCUI Shell : PowerShell

- PCの設定や管理などで、バッチ的に処理したいというニーズは大きい。また、Windows OS内部で、資源をオブジェクトで管理しているのに、command操作時に、テキストベースでしか情報を取得できないのは、実際に管理の面から
見て、不自由となってきた。

- より、Microsoft社は、次世代シェルとして、powershellを2006/11月に提供した。その後、Windows 8.1からはWindowsに、Powershellは標準機能としてインストールされている。標準のCUI Shellとして既に10年たち、2019/10現在、PowerShellのバージョンは、version5.1となった。

- PowerShellは、仮想実行環境である.NET Framework上で動作し、コンソールより、コマンド（cmdletと呼ぶ）により、OSのリソースを取得したり、操作したりすることができる。Windows OSのGUI上でできる操作は、すべてコマンドラインでも操作可能である。


- 以下、NetIPAddressオブジェクトを取得する例を記述する。

#### `Get-NetIPAddress` Cmdlet

In [4]:
Get-NetIPAddress -AddressFamily IPv4 | where {$_.PrefixOrigin -eq 'Dhcp'} |select -first 1



IPAddress         : 192.168.1.38
InterfaceIndex    : 3
InterfaceAlias    : ■C■[■T■l■b■g
AddressFamily     : IPv4
Type              : Unicast
PrefixLength      : 24
PrefixOrigin      : Dhcp
SuffixOrigin      : Dhcp
AddressState      : Preferred
ValidLifetime     : 02:57:39
PreferredLifetime : 02:57:39
SkipAsSource      : False
PolicyStore       : ActiveStore





- 上記が、`Get-IPAddress` cmdletで、情報を取得した例である。取得したオブジェクトは、PowerShell内でデフォルトの形式に自動でフォーマッティングされて、コンソール上に出力される。

- 取得しているオブジェクトを、以下のコマンドで確認する。


#### 取得したObjectの確認

##### GetType()で、Objectの型を確認

- 以下、`Microsoft.Management.Infrastructure.CimInstance`というオブジェクト型であることが分かる。

In [5]:
Get-NetIPAddress -AddressFamily IPv4 | where {$_.PrefixOrigin -eq 'Dhcp'} |select -first 1 | set -Name ipaddrObj

In [6]:
$ipaddrObj.GetType() | Select module, Fullname, Name | fl 



Module   : Microsoft.Management.Infrastructure.dll
FullName : Microsoft.Management.Infrastructure.CimInstance
Name     : CimInstance





##### Get-MemberでObjectのProperty、Methodを確認

- 以下、対象オブジェクトのMemberを確認する。

In [7]:
$ipaddrObj | Get-Member -MemberType Property



   TypeName: Microsoft.Management.Infrastructure.CimInstance#ROOT/StandardCimv2/MSFT_NetIPAddress

Name                     MemberType Definition                                           
----                     ---------- ----------                                           
Address                  Property   string Address {get;set;}                            
AddressOrigin            Property   uint16 AddressOrigin {get;set;}                      
AddressType              Property   uint16 AddressType {get;set;}                        
AvailableRequestedStates Property   uint16[] AvailableRequestedStates {get;set;}         
Caption                  Property   string Caption {get;set;}                            
CommunicationStatus      Property   uint16 CommunicationStatus {get;set;}                
CreationClassName        Property   string CreationClassName {get;set;}                  
Description              Property   string Description {get;set;}                        

In [8]:
$ipaddrObj | Get-Member -MemberType Method



   TypeName: Microsoft.Management.Infrastructure.CimInstance#ROOT/StandardCimv2/MSFT_NetIPAddress

Name                      MemberType Definition                                                                        
----                      ---------- ----------                                                                        
Clone                     Method     System.Object ICloneable.Clone()                                                  
Dispose                   Method     void Dispose(), void IDisposable.Dispose()                                        
Equals                    Method     bool Equals(System.Object obj)                                                    
GetCimSessionComputerName Method     string GetCimSessionComputerName()                                                
GetCimSessionInstanceId   Method     guid GetCimSessionInstanceId()                                                    
GetHashCode               Method     int GetHashCode()     

##### Format-List CmdletでObjectの内容を確認

- 以下、対象オブジェクトのデータを詳細に確認する。

In [9]:
$ipaddrObj | Format-List *



PrefixOrigin             : Dhcp
SuffixOrigin             : Dhcp
Type                     : Unicast
Store                    : ActiveStore
AddressFamily            : IPv4
AddressState             : Preferred
ifIndex                  : 3
Caption                  : 
Description              : 
ElementName              : 
InstanceID               : 
CommunicationStatus      : 
DetailedStatus           : 
HealthState              : 
InstallDate              : 
Name                     : ;C<8;@B8;8=B55=55;55;
OperatingStatus          : 
OperationalStatus        : 
PrimaryStatus            : 
Status                   : 
StatusDescriptions       : 
AvailableRequestedStates : 
EnabledDefault           : 2
EnabledState             : 
OtherEnabledState        : 
RequestedState           : 12
TimeOfLastStateChange    : 
TransitioningToState     : 12
CreationClassName        : 
SystemCreationClassName  : 
SystemName               : 
NameFormat               : 
OtherTypeDescription     : 
Protocol

以上